In [1]:
# Imports
from tkinter import *
from tkinter.ttk import * 
import datetime
import pandas as pd

In [2]:
cols = [i for i in range(0,70)]

In [80]:
pdread = pd.read_csv('./contract.txt' , sep='|' , skiprows=0 , names=cols)

In [81]:
pdread[7]= pdread[7]/100
pdread[7] = pdread[7].astype('int')

In [83]:
script_list =sorted(pdread[3].dropna().unique().tolist())
strategy_list = ['Butterfly']
option_type_list = sorted(['CE' , 'PE'])
ratio_list = ['121' ,'143']

In [79]:
root  = Tk()
root.geometry('650x400')
root.title('Codesure')

def c1_selected(event):
    script_name = event.widget.get()
    global duplicate
    duplicate = pdread[pdread[3]==script_name]
    c2['values'] = sorted(duplicate[2].dropna().unique().tolist())
    
def c2_selected(event):
    instrument = event.widget.get()
    global duplicate
    duplicate = duplicate[duplicate[2] == instrument]

def c3_selected(event):
    option = event.widget.get()
    global duplicate
    duplicate = duplicate[duplicate[8] == option]
    expiryList = duplicate[6].tolist()
    comboExpiry = []
    for i in expiryList:
        comboExpiry.append(datetime.datetime.fromtimestamp(i).strftime('%d-%m-%Y'))
        
    duplicate[6] = comboExpiry
    c4['values'] = sorted(np.unique(comboExpiry).tolist())
    
def c4_selected(event):
    exp = event.widget.get()
    global duplicate
    duplicate = duplicate[duplicate[6]==exp]
    c5['values'] = sorted(duplicate[7].tolist())

def c5_selected(event):
    global val
    val = event.widget.get()
    l = c5['values']
    new_list = l[l.index(val)+1:]
    c6['values'] = new_list
    
def c6_selected(event):
    global duplicate
    duplicate = duplicate[duplicate[7]>= int(val)]
    duplicate = duplicate.reset_index()
    end = duplicate[duplicate[7] == int(event.widget.get())].index
    duplicate = duplicate[:end[0]]
    
def c10_selected(event):
    global ratio
    ratio = event.widget.get()
    
def submit():
    gap = gap1.get()
    center = center1.get()
    sst = sst1.get()
    global ratio
    global duplicate
    duplicate = duplicate.sort_values(by=[7]).reset_index().drop(['index','level_0'] ,axis=1)

    if sst == 'Butterfly':
        ratio = [int(d) for d in ratio]
        start = duplicate.loc[0][7]
        n = len(duplicate)-1
        end = duplicate.loc[n][7]
    
        l11.insert(END , f'SPOT   BS    SS   MAX \n')
        while start <= end:
#             strike.append(start-center)
            bid = [np.random.randint(10,30) for _ in range(3)]
            ask = [np.random.randint(bid[x]-1 , bid[x]+1) for x in range(3)]
            bs = ask[0] * r[0] - bid[1] * r[1] + ask[2] * r[2] 
            ss = -(bid[0] * r[0] + ask[1]* r[1] + bid[2] * r[2])
            l11.insert(END , f'{start}   {bs}   {ss}   {max(bs , ss)} \n')
#             strike.append(start)
#             strike.append(start+center)
#             l11.insert(END ,f'{start - center} * {ratio[0]} Buy \n' )
#             l11.insert(END ,f'{start} * {ratio[1]} Sell \n' )
#             l11.insert(END ,f'{start + center} * {ratio[2]} Buy \n\n' )
            
            start += gap
l1 = Label(root , text='Script')
c1 = Combobox(root )
c1['values'] = script_list

c1.bind('<<ComboboxSelected>>' , c1_selected)

l2 = Label(root , text='Instrument Name')
c2 = Combobox(root )
c2.bind('<<ComboboxSelected>>' ,c2_selected)

l3 = Label(root , text='Option Type')
c3 = Combobox(root)
c3['values'] = option_type_list
c3.bind('<<ComboboxSelected>>' , c3_selected)

l4 = Label(root , text='Expiry' )
c4 = Combobox(root )
c4.bind('<<ComboboxSelected>>' , c4_selected)

l5 = Label(root , text='Lower Range')
c5 = Combobox(root)

c5.bind('<<ComboboxSelected>>' , c5_selected)

l6 = Label(root , text='Upper Range')
c6 = Combobox(root)
c6.bind('<<ComboboxSelected>>' , c6_selected)

gap1 = IntVar()
l7 = Label(root , text='Gap')
c7 = Entry(root , textvariable=gap1)

center1 = IntVar()
l8 = Label(root , text='Center')
c8 = Entry(root , textvariable=center1)

sst1 = StringVar()
l9 = Label(root , text= 'Strategy')
c9 = Combobox(root , values = strategy_list , textvariable=sst1)

l10 = Label(root , text='Ratio')
c10 = Combobox(root ,values= ratio_list)

c10.bind('<<ComboboxSelected>>' , c10_selected)

btn = Button(root ,text='Submit', command=submit)

l11 = Text(root )

l1.grid(column=0 , row=0)
c1.grid(column=1 , row=0)

l2.grid(column=2 , row=0)
c2.grid(column=3 , row=0)

l3.grid(column=0 , row=1)
c3.grid(column=1 , row=1)

l4.grid(column=2 , row=1)
c4.grid(column=3 , row=1)

l5.grid(column=0 , row=2)
c5.grid(column=1 , row=2)

l6.grid(column=2 , row=2)
c6.grid(column=3 , row=2)

l9.grid(column=0 , row=3)
c9.grid(column=1 , row=3)

l8.grid(column=2 , row=3)
c8.grid(column=3 , row=3)

l7.grid(column=0 , row=4)
c7.grid(column=1 , row=4)

l10.grid(column=2 , row=4)
c10.grid(column=3 , row=4)

btn.grid(column=2 , row=5)

l11.grid(column=0 , row=6 , columnspan=4)
root.mainloop()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [84]:

duplicate

,level_0,0,1,2,3,4,5,6,7,8,...,60,61,62,63,64,65,66,67,68,69
0,0,35323,26000,OPTIDX,NIFTY,XX,NaN,09-04-2011,13050,PE,...,N,N,N,C,N,NaN,NaN,1530,N,NaN
1,1,35330,26000,OPTIDX,NIFTY,XX,NaN,09-04-2011,13100,PE,...,N,N,N,C,N,NaN,NaN,1625,N,NaN
2,2,35332,26000,OPTIDX,NIFTY,XX,NaN,09-04-2011,13150,PE,...,N,N,N,C,N,NaN,NaN,1900,N,NaN
3,3,43570,26000,OPTIDX,NIFTY,XX,NaN,09-04-2011,16450,PE,...,N,N,N,C,N,NaN,NaN,210370,N,NaN
4,4,43572,26000,OPTIDX,NIFTY,XX,NaN,09-04-2011,16500,PE,...,N,N,N,C,N,NaN,NaN,215335,N,NaN
5,5,43574,26000,OPTIDX,NIFTY,XX,NaN,09-04-2011,16550,PE,...,N,N,N,C,N,NaN,NaN,220305,N,NaN
6,6,43576,26000,OPTIDX,NIFTY,XX,NaN,09-04-2011,16600,PE,...,N,N,N,C,N,NaN,NaN,225275,N,NaN
